In [1]:
import pandas as pd
from utils import make_connection

In [2]:
db_conn = make_connection(config_file = 'db_config.ini')
db_cursor = db_conn.cursor()

Connection successful


In [3]:
wh_conn = make_connection(config_file = 'dw_config.ini')
wh_cursor = wh_conn.cursor()

Connection successful


**User_Dim**

In [4]:
# Fetching all_restaurants table data
all_users_data = 'Select * from users'

db_cursor.execute(all_users_data)

In [5]:
rows  = db_cursor.fetchall()
count = db_cursor.rowcount
print(count)

500


In [6]:
# Creating dataframe of all_restaurants data
all_users_df = pd.DataFrame(rows)
all_users_df.head(3)

,0,1,2,3,4,5
0,Miss Maria Meza MD,christopherobrien@hotmail.com,688662959,Admin,1,HS6+7RXrp6
1,Eileen Pierce,zgardner@alvarado-patton.com,1814255960,Admin,2,T&gb5UeTcm
2,Kenneth Calderon,mckenziejerry@davis-garcia.com,9595904348,Customer,3,*6sD0y&is7


In [7]:
num_fields = len(db_cursor.description)
field_names = [i[0] for i in db_cursor.description]
field_names

['Name', 'Email', 'Phone', 'Role', 'User_ID', 'Password']

In [8]:
all_users_df.columns = field_names
all_users_df.head()

,Name,Email,Phone,Role,User_ID,Password
0,Miss Maria Meza MD,christopherobrien@hotmail.com,688662959,Admin,1,HS6+7RXrp6
1,Eileen Pierce,zgardner@alvarado-patton.com,1814255960,Admin,2,T&gb5UeTcm
2,Kenneth Calderon,mckenziejerry@davis-garcia.com,9595904348,Customer,3,*6sD0y&is7
3,Margaret Nelson,kristen38@hansen-smith.org,563094164,Customer,4,kR(82)Lfq$
4,Lisa Willis,kennethmccoy@campbell.info,2190028180,Customer,5,de3SVPcom+


In [9]:
users_df = all_users_df[['User_ID', 'Name', 'Email']]

In [10]:
for i, row in users_df.iterrows():
    sql = "INSERT INTO user_dim (user_id, name, email) VALUES (%s, %s, %s)"
    wh_cursor.execute(sql, tuple(row))

wh_conn.commit()

**Table_Dim**


In [11]:
# Fetching all_restaurants table data
all_tables_data = 'Select * from table_details'

db_cursor.execute(all_tables_data)

rows  = db_cursor.fetchall()
count = db_cursor.rowcount
print(count)

20


In [12]:
# Creating dataframe of all_restaurants data
all_tables_df = pd.DataFrame(rows)

num_fields = len(db_cursor.description)
field_names = [i[0] for i in db_cursor.description]
field_names

all_tables_df.columns = field_names
all_tables_df.head()

,Table_ID,Seating_Capacity,Location
0,1,2,Indoor Dining
1,2,7,Indoor Dining
2,3,3,Patio
3,4,7,Patio
4,5,7,Indoor Dining


In [13]:
tables_df = all_tables_df[['Table_ID', 'Seating_Capacity', 'Location']]

In [14]:
for i, row in tables_df.iterrows():
    sql = "INSERT INTO tables_dim (table_id, seating_capacity, location) VALUES (%s, %s, %s)"
    wh_cursor.execute(sql, tuple(row))

wh_conn.commit()

**Menu_Dim**

In [15]:
# Fetching all_restaurants table data
all_menu_data = '''
                SELECT  m.Menu_ID, m.Category, m.Item_Name, m.Price, ROUND(AVG(r.Rating),2) AS Rating
                FROM star_spartans_db2.menu m
                JOIN star_spartans_db2.review r on m.Menu_ID = r.Menu_ID
                GROUP BY m.Menu_ID;
                '''

db_cursor.execute(all_menu_data)

rows  = db_cursor.fetchall()
count = db_cursor.rowcount
print(count)

50


In [16]:
# Creating dataframe of all_restaurants data
all_menu_df = pd.DataFrame(rows)

num_fields = len(db_cursor.description)
field_names = [i[0] for i in db_cursor.description]
field_names

all_menu_df.columns = field_names
all_menu_df.head()

,Menu_ID,Category,Item_Name,Price,Rating
0,1,Appetizer,Samosas,4.99,3.50
1,2,Appetizer,Pakoras (vegetable fritters),5.99,3.50
2,3,Appetizer,Aloo Tikki (potato patties),4.49,3.67
3,4,Appetizer,"Chaat (varieties like papdi chaat, bhel puri)",6.99,3.50
4,5,Appetizer,Paneer Tikka,8.99,2.00


In [17]:
for i, row in all_menu_df.iterrows():
    sql = "INSERT INTO menu_dim (menu_id, category, item_name, price, rating) VALUES (%s, %s, %s, %s, %s)"
    wh_cursor.execute(sql, tuple(row))

wh_conn.commit()

**Date_Dim**

In [18]:
# Fetching all_restaurants table data
all_orders_data = 'Select * from orders'

db_cursor.execute(all_orders_data)

rows  = db_cursor.fetchall()
count = db_cursor.rowcount
print(count)

6000


In [19]:
# Creating dataframe of all_restaurants data
all_orders_df = pd.DataFrame(rows)

num_fields = len(db_cursor.description)
field_names = [i[0] for i in db_cursor.description]
field_names

all_orders_df.columns = field_names
all_orders_df.head()

,Order_ID,Day_Of_week,Month,Year,Date,Order_Time,User_ID
0,1,5,9,2021,11,22,18
1,2,4,3,2023,12,21,483
2,3,3,9,2022,14,7,138
3,4,6,9,2023,14,10,120
4,5,6,5,2022,4,14,407


In [20]:
orders_df = all_orders_df[['Date', 'Month', 'Year', 'Day_Of_week']]
orders_df = orders_df.rename(columns={'Day_Of_week': 'Day_Of_Week'})
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Date         6000 non-null   int64
 1   Month        6000 non-null   int64
 2   Year         6000 non-null   int64
 3   Day_Of_Week  6000 non-null   int64
dtypes: int64(4)
memory usage: 187.6 KB


Inserting reservation dates now

In [21]:
# Fetching all_restaurants table data
all_reservations_data = 'Select * from reservation'

db_cursor.execute(all_reservations_data)

rows  = db_cursor.fetchall()
count = db_cursor.rowcount
print(count)

50


In [22]:
# Creating dataframe of all_restaurants data
all_reservations_df = pd.DataFrame(rows)

num_fields = len(db_cursor.description)
field_names = [i[0] for i in db_cursor.description]
field_names

all_reservations_df.columns = field_names
all_reservations_df.head()

,Reservation_ID,Day_Of_Week,Date,Reservation_Time,Number_of_Guests,User_ID,Table_ID
0,1,5,2023-02-20,16,5,396,2
1,2,2,2023-02-09,13,1,395,14
2,3,1,2023-06-02,15,8,365,20
3,4,5,2023-11-28,11,6,354,5
4,5,5,2023-01-31,13,2,482,8


In [23]:
reservations_df = all_reservations_df[['Date', 'Day_Of_Week']]
reservations_df.head()

,Date,Day_Of_Week
0,2023-02-20,5
1,2023-02-09,2
2,2023-06-02,1
3,2023-11-28,5
4,2023-01-31,5


In [24]:
reservations_df['Date'] = pd.to_datetime(reservations_df['Date'])

# Extract year, month, and day
reservations_df['Year'] = reservations_df['Date'].dt.year
reservations_df['Month'] = reservations_df['Date'].dt.month
reservations_df['Date'] = reservations_df['Date'].dt.day

reservations_df.head()

C:\Users\Parth\AppData\Local\Temp\ipykernel_15984\1324032758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reservations_df['Date'] = pd.to_datetime(reservations_df['Date'])
C:\Users\Parth\AppData\Local\Temp\ipykernel_15984\1324032758.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reservations_df['Year'] = reservations_df['Date'].dt.year
C:\Users\Parth\AppData\Local\Temp\ipykernel_15984\1324032758.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

,Date,Day_Of_Week,Year,Month
0,20,5,2023,2
1,9,2,2023,2
2,2,1,2023,6
3,28,5,2023,11
4,31,5,2023,1


In [25]:
cleaned_reservations_df = reservations_df[['Date', 'Month', 'Year', 'Day_Of_Week']]
cleaned_reservations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Date         50 non-null     int64
 1   Month        50 non-null     int64
 2   Year         50 non-null     int64
 3   Day_Of_Week  50 non-null     int64
dtypes: int64(4)
memory usage: 1.7 KB


In [26]:
# Concatenate the two DataFrames
combined_order_reservation_date_df = pd.concat([orders_df, cleaned_reservations_df])

In [27]:
combined_order_reservation_date_df.head()

,Date,Month,Year,Day_Of_Week
0,11,9,2021,5
1,12,3,2023,4
2,14,9,2022,3
3,14,9,2023,6
4,4,5,2022,6


In [28]:
combined_order_reservation_date_df = combined_order_reservation_date_df.drop_duplicates(subset=['Date', 'Month', 'Year'])

In [29]:
for i, row in combined_order_reservation_date_df.iterrows():
    sql = "INSERT INTO date_dim (day, month, year, dow) VALUES (%s, %s, %s, %s)"
    wh_cursor.execute(sql, tuple(row))

wh_conn.commit()

**Reservation_Fact**

In [30]:
all_reservations_df.head()

,Reservation_ID,Day_Of_Week,Date,Reservation_Time,Number_of_Guests,User_ID,Table_ID
0,1,5,2023-02-20,16,5,396,2
1,2,2,2023-02-09,13,1,395,14
2,3,1,2023-06-02,15,8,365,20
3,4,5,2023-11-28,11,6,354,5
4,5,5,2023-01-31,13,2,482,8


In [31]:
reservations_fact_df = all_reservations_df[['Reservation_ID', 'Number_of_Guests', 'Reservation_Time', 'Date', 'Table_ID', 'User_ID',]]
reservations_fact_df.head()

,Reservation_ID,Number_of_Guests,Reservation_Time,Date,Table_ID,User_ID
0,1,5,16,2023-02-20,2,396
1,2,1,13,2023-02-09,14,395
2,3,8,15,2023-06-02,20,365
3,4,6,11,2023-11-28,5,354
4,5,2,13,2023-01-31,8,482


In [32]:
reservations_fact_df['Date'] = pd.to_datetime(reservations_fact_df['Date'])

# Extract year, month, and day
reservations_fact_df['date_id_year'] = reservations_fact_df['Date'].dt.year
reservations_fact_df['date_id_month'] = reservations_fact_df['Date'].dt.month
reservations_fact_df['date_id_day'] = reservations_fact_df['Date'].dt.day

reservations_fact_df.head()

C:\Users\Parth\AppData\Local\Temp\ipykernel_15984\82484598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reservations_fact_df['Date'] = pd.to_datetime(reservations_fact_df['Date'])


,Reservation_ID,Number_of_Guests,Reservation_Time,Date,Table_ID,User_ID,date_id_year,date_id_month,date_id_day
0,1,5,16,2023-02-20,2,396,2023,2,20
1,2,1,13,2023-02-09,14,395,2023,2,9
2,3,8,15,2023-06-02,20,365,2023,6,2
3,4,6,11,2023-11-28,5,354,2023,11,28
4,5,2,13,2023-01-31,8,482,2023,1,31


In [33]:
cleaned_reservations_fact_df = reservations_fact_df[['Reservation_ID', 'Number_of_Guests', 'Reservation_Time', 'date_id_day', 'date_id_month', 'date_id_year', 'Table_ID', 'User_ID',]]

cleaned_reservations_fact_df.head()

,Reservation_ID,Number_of_Guests,Reservation_Time,date_id_day,date_id_month,date_id_year,Table_ID,User_ID
0,1,5,16,20,2,2023,2,396
1,2,1,13,9,2,2023,14,395
2,3,8,15,2,6,2023,20,365
3,4,6,11,28,11,2023,5,354
4,5,2,13,31,1,2023,8,482


In [34]:
for i, row in cleaned_reservations_fact_df.iterrows():
    sql = '''
            INSERT INTO reservation_fact (reservation_id, no_of_guest, time, date_id_day, date_id_month, date_id_year, table_id, user_id) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        '''
    wh_cursor.execute(sql, tuple(row))

wh_conn.commit()

**Sales_Fact**

In [35]:
all_orders_df.head()

,Order_ID,Day_Of_week,Month,Year,Date,Order_Time,User_ID
0,1,5,9,2021,11,22,18
1,2,4,3,2023,12,21,483
2,3,3,9,2022,14,7,138
3,4,6,9,2023,14,10,120
4,5,6,5,2022,4,14,407


In [36]:
# Fetching all_restaurants table data
all_order_details_data = 'Select * from order_details'

db_cursor.execute(all_order_details_data)

rows  = db_cursor.fetchall()
count = db_cursor.rowcount
print(count)

10000


In [37]:
# Creating dataframe of all_restaurants data
all_order_details_df = pd.DataFrame(rows)

num_fields = len(db_cursor.description)
field_names = [i[0] for i in db_cursor.description]
field_names

all_order_details_df.columns = field_names
all_order_details_df.head()

,Order_Details_ID,Unit_Price,Quantity,Menu_ID,Order_ID
0,1,14.99,10,10,4388
1,2,7.49,2,31,2311
2,3,12.99,5,16,971
3,4,5.99,7,4,3947
4,5,10.99,9,17,4598


In [38]:
# Assuming all_orders_df and all_order_details_df are your DataFrames
merged_order_details_df = all_orders_df.merge(all_order_details_df, on='Order_ID', how='inner')
merged_order_details_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order_ID          10000 non-null  int64  
 1   Day_Of_week       10000 non-null  int64  
 2   Month             10000 non-null  int64  
 3   Year              10000 non-null  int64  
 4   Date              10000 non-null  int64  
 5   Order_Time        10000 non-null  int64  
 6   User_ID           10000 non-null  int64  
 7   Order_Details_ID  10000 non-null  int64  
 8   Unit_Price        10000 non-null  float64
 9   Quantity          10000 non-null  int64  
 10  Menu_ID           10000 non-null  int64  
dtypes: float64(1), int64(10)
memory usage: 937.5 KB


In [39]:
merged_order_details_df['total_sale_amount'] = merged_order_details_df['Unit_Price'] * merged_order_details_df['Quantity']

In [40]:
merged_order_details_df.head()

,Order_ID,Day_Of_week,Month,Year,Date,Order_Time,User_ID,Order_Details_ID,Unit_Price,Quantity,Menu_ID,total_sale_amount
0,1,5,9,2021,11,22,18,2911,13.99,5,6,69.95
1,1,5,9,2021,11,22,18,9625,9.99,4,5,39.96
2,2,4,3,2023,12,21,483,6132,5.49,9,35,49.41
3,3,3,9,2022,14,7,138,2271,14.99,7,46,104.93
4,3,3,9,2022,14,7,138,3742,14.99,10,22,149.90


In [41]:
# Convert to string and pad
merged_order_details_df['Order_ID'] = merged_order_details_df['Order_ID'].astype(str).str.zfill(5)
merged_order_details_df['Order_Details_ID'] = merged_order_details_df['Order_Details_ID'].astype(str).str.zfill(5)

merged_order_details_df.head()

,Order_ID,Day_Of_week,Month,Year,Date,Order_Time,User_ID,Order_Details_ID,Unit_Price,Quantity,Menu_ID,total_sale_amount
0,00001,5,9,2021,11,22,18,02911,13.99,5,6,69.95
1,00001,5,9,2021,11,22,18,09625,9.99,4,5,39.96
2,00002,4,3,2023,12,21,483,06132,5.49,9,35,49.41
3,00003,3,9,2022,14,7,138,02271,14.99,7,46,104.93
4,00003,3,9,2022,14,7,138,03742,14.99,10,22,149.90


In [42]:
cleaned_sales_fact_df = merged_order_details_df[['Quantity', 'total_sale_amount', 'Order_ID', 'Order_Details_ID', 'Menu_ID', 'User_ID', 'Date', 'Month', 'Year']]

cleaned_sales_fact_df.head()

,Quantity,total_sale_amount,Order_ID,Order_Details_ID,Menu_ID,User_ID,Date,Month,Year
0,5,69.95,00001,02911,6,18,11,9,2021
1,4,39.96,00001,09625,5,18,11,9,2021
2,9,49.41,00002,06132,35,483,12,3,2023
3,7,104.93,00003,02271,46,138,14,9,2022
4,10,149.90,00003,03742,22,138,14,9,2022


In [43]:
for i, row in cleaned_sales_fact_df.iterrows():
    sql = '''
            INSERT INTO sales_fact (quantity, total_sale_amount, order_id, order_details_id, menu_id, user_id, date_id_day, date_id_month, date_id_year) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''
    wh_cursor.execute(sql, tuple(row))

wh_conn.commit()

In [44]:
wh_cursor.close()
wh_conn.close()

In [45]:
db_cursor.close()
db_conn.close()